In [ ]:
import numpy as np
import matplotlib as mpl
mpl.use('pgf')

def figsize(scale):
    fig_width_pt = 469.755                          # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = fig_width*golden_mean              # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size

pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True,                # use LaTeX to write all text
    "font.family": "serif",
    "font.serif": [],                   # blank entries should cause plots to inherit fonts from the document
    "font.sans-serif": [],
    "font.monospace": [],
    "axes.labelsize": 10,               # LaTeX default is 10pt font.
    "font.size": 10,
    "legend.fontsize": 8,               # Make the legend/label fonts a little smaller
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "figure.figsize": figsize(0.9),     # default fig size of 0.9 textwidth
    "pgf.preamble": [
        r"\usepackage[utf8x]{inputenc}",    # use utf8 fonts becasue your computer can handle it :)
        r"\usepackage[T1]{fontenc}",        # plots will be generated using this preamble
        ]
    }
mpl.rcParams.update(pgf_with_latex)

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# I make my own newfig and savefig functions
def newfig(width):
    plt.clf()
    fig = plt.figure(figsize=figsize(width))
    ax = fig.add_subplot(111)
    return fig, ax

def savefig(filename):
    plt.savefig('{}.pgf'.format(filename))
    plt.savefig('{}.pdf'.format(filename))


# Simple plot
fig, ax  = newfig(0.6)

def ema(y, a):
    s = []
    s.append(y[0])
    for t in range(1, len(y)):
        s.append(a * y[t] + (1-a) * s[t-1])
    return np.array(s)
    
y = [0]*200
y.extend([20]*(1000-len(y)))
s = ema(y, 0.01)

ax.plot(s)
ax.set_xlabel('X Label')
ax.set_ylabel('EMA')

savefig('ema')

In [ ]:
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import json

def microseconds(x, pos):
    'The two args are the value and tick position'
    return '%i µs' % x

def milliseconds(x, pos):
    'The two args are the value and tick position'
    return '%i ms' % (x*1e-3)

def show_boxplot(data):
    formatter = FuncFormatter(microseconds)
    plot_labels = ['None', '2', '4', '8', '16'][0:len(data)]
    fig, ax = plt.subplots()
    ax.yaxis.set_major_formatter(formatter)
    plt.boxplot(data, labels=plot_labels, showfliers=False)
    #plt.title('Quotient filter vs. dictionary')
    plt.xlabel('remainder size')
    plt.ylabel('run time')
    ax.set_ylim(ymin=0)
    plt.savefig('{}.pgf'.format('output'))
    plt.savefig('{}.pdf'.format('output'))
    plt.show()
    
def big_plot(value_column, quotient_filter, dictionary, art): 
    formatter = FuncFormatter(milliseconds)
    plot_labels = ['Value Column', 'Dictionary', 'Filter', 'ART']
    fig, ax = plt.subplots()
    ax.yaxis.set_major_formatter(formatter)
    data = [value_column, dictionary, quotient_filter, art]
    plt.boxplot(data, labels=plot_labels, showfliers=False)
    plt.title('Table Scan Runtime: Integer Scan')
    plt.xlabel('remainder size')
    plt.ylabel('run time')
    ax.set_ylim(ymin=0)
    plt.savefig('{}.pgf'.format('output'))
    plt.savefig('{}.pdf'.format('output'))
    plt.show()
        
def get_column_names(): 
    table_info = json.load(open('benchmark_results.csv.json','r'))
    #print(json.dumps(table_info, indent=2, sort_keys=False))
    column_names = []
    for column in table_info["columns"]:
        column_names.append(column["name"])
    return column_names
    
def get_data():
    return pd.read_csv('benchmark_results.csv', header=None, names=get_column_names())
        
def plot_all_groups():
    group_columns = get_column_names()
    group_columns.remove('remainder_size')
    group_columns.remove('run_time')
    grouped_data = get_data().groupby(group_columns)
    for group_info in list(grouped_data.groups):
        for i in range(0, len(group_columns)):
            print(group_columns[i] + ": " + str(group_info[i]))
        sub_groups = grouped_data.get_group(group_info).groupby('remainder_size')
        run_time_data = []
        for remainder_size in list(sub_groups.groups):
            run_time_data.append(sub_groups.get_group(remainder_size)['run_time'])   
        show_boxplot(run_time_data)

def select(data, data_type,remainder_size, dictionary, btree, art):
    return data[(data.data_type == data_type) &
                (data.remainder_size == remainder_size) & 
                (data.dictionary == dictionary) & 
                (data.btree == btree) & 
                (data.art == art)]['run_time']

    

In [ ]:
import pandas as pd

data = get_data()
print(get_column_names())
art_data          = select(data, data_type='int', remainder_size=0, dictionary=1, btree=0, art=1)
filter_data       = select(data, data_type='int', remainder_size=4, dictionary=0, btree=0, art=0)
dictionary_data   = select(data, data_type='int', remainder_size=0, dictionary=1, btree=0, art=0)
value_column_data = select(data, data_type='int', remainder_size=0, dictionary=0, btree=0, art=0)
big_plot(value_column_data, filter_data, dictionary_data, art_data)

art_data          = select(data, data_type='string', remainder_size=0, dictionary=1, btree=0, art=1)
filter_data       = select(data, data_type='string', remainder_size=4, dictionary=0, btree=0, art=0)
dictionary_data   = select(data, data_type='string', remainder_size=0, dictionary=1, btree=0, art=0)
value_column_data = select(data, data_type='string', remainder_size=0, dictionary=0, btree=0, art=0)
big_plot(value_column_data, filter_data, dictionary_data, art_data)

#plot_all_groups()
